In [23]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification,BertConfig

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss

import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.utils import pad_sequences

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

from keras.layers import Embedding


In [24]:
config = BertConfig.from_pretrained('prajjwal1/bert-mini')
model = TFBertForSequenceClassification(config=config)


In [25]:
model.build()

In [26]:
model.load_weights("weights-bert.h5")

In [27]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  11170560  
                                                                 
 dropout_27 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  514       
                                                                 
Total params: 11171074 (42.61 MB)
Trainable params: 11171074 (42.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
data_processed = pd.read_csv("../data/"+"labelling_dataset_v1.csv")
# Split into training and testing data
X = data_processed["text_processed"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

# create the model
model = Sequential()

model.add(Embedding(input_dim=vocab_size+1, output_dim=embedding_size))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.LSTM(100))
model.add(layers.Dense(6, activation='sigmoid'))

# Students will be ending their code here

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Change the number of epochs and the batch size depending on the RAM Size

es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
    epochs=150,
    batch_size=128,
    callbacks=[es], validation_split=0.3
)
res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')
print('Testing loss \t', res[0]*100)
print('Testing accuracy ', res[1]*100)


FileNotFoundError: [Errno 2] No such file or directory: '../data/labelling_dataset_v1.csv'

In [29]:
y_true = y_test
y_pred=model.predict(X_test_pad)

h3 = hamming_loss(y_true, y_pred)
h3

NameError: name 'y_test' is not defined